In [3]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


In [4]:
df = pd.read_csv('datasets/comp_data3.csv')
df.head()

,all_info_combined,all_info_combined_clean,brand,category,description,name,price,store
0,"Organic Valley, Organic Half & Half, Ultra Pas...","organic valley, organic half & half, ultra pas...",Organic Valley,Dairy,When cows spend their days on lush green pastu...,"Organic Valley, Organic Half & Half, Ultra Pas...",4.69,Whole Foods Market
1,"Organic Valley, Organic Half & Half, Ultra Pas...","organic valley, organic half & half, ultra pas...",Organic Valley,Dairy,When cows spend their days on lush green pastu...,"Organic Valley, Organic Half & Half, Ultra Pas...",2.69,Amazon
2,"Organic Valley, Organic Heavy Whipping Cream, ...","organic valley, organic heavy whipping cream, ...",Organic Valley,Dairy,When cows spend their days on lush green pastu...,"Organic Valley, Organic Heavy Whipping Cream, ...",4.49,Amazon
3,Folgers Gourmet Selections Lively Colombian Co...,folgers gourmet selections lively colombian co...,Folgers Gourmet Selections Lively Colombian Co...,Dairy,Folgers Gourmet Selections K-Cups - Med - Colu...,Folgers Gourmet Selections Lively Colombian Co...,9.06,Amazon
4,"Lactaid 2% Reduced Fat Milk, 64 fl oz You won’...","lactaid 2% reduced fat milk, 64 fl oz won’t be...",Lactaid,Dairy,You won’t believe it’s 100% lactose free. Get ...,"Lactaid 2% Reduced Fat Milk, 64 fl oz",3.99,Amazon


In [5]:
df.isna().sum()

all_info_combined             0
all_info_combined_clean       0
brand                         0
category                      0
description                2840
name                          0
price                       295
store                         0
dtype: int64

In [6]:
df.dropna(how = 'any',inplace=True,subset=['price'])
 

In [7]:
from gensim.test.utils import common_texts
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [8]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['all_info_combined_clean'])]



In [9]:
documents

[TaggedDocument(words="organic valley, organic half & half, ultra pasteurized, quart, 32 oz cows spend days lush green pastures, get smooth texture rich flavor found organic valley's organic half & half. creamy, delicious, award-winning organic half & half made two - organic milk organic cream pasture-raised cows. organic valley organic half & half perfect coffee, tea, hot chocolate elevate favorite soups recipes - almost always substitute half & half milk want add extra flavor creaminess.", tags=[0]),
 TaggedDocument(words="organic valley, organic half & half, ultra pasteurized, pint, 16 oz cows spend days lush green pastures, get smooth texture rich flavor found organic valley's organic half & half. creamy, delicious, award-winning organic half & half made two - organic milk organic cream pasture-raised cows. organic valley organic half & half perfect coffee, tea, hot chocolate elevate favorite soups recipes - almost always substitute half & half milk want add extra flavor creaminess

In [2]:
model =Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

NameError: name 'Doc2Vec' is not defined

In [13]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens


tagged = df.apply(lambda r: TaggedDocument(tags= [r['name']],words=tokenize_text(r['all_info_combined_clean'])), axis=1)



In [14]:
model = Doc2Vec(alpha=0.025, min_alpha=0.025)
model.build_vocab(tagged)
for epoch in range(10):
    model.train(tagged, total_examples=model.corpus_count, epochs=model.epochs)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
    

In [ ]:
model = Doc2Vec(tagged,dm=0, alpha =0.01,size=10, min_alpha=0.025, min_count=0)
model_dbow.train(tagged,end_alpha =0.025,total_examples=model.corpus_count,epochs=2)

In [15]:
model.wv.vocab.keys()

dict_keys(['organic', 'valley', 'half', 'ultra', 'pasteurized', 'quart', '32', 'oz', 'cows', 'spend', 'days', 'lush', 'green', 'pastures', 'get', 'smooth', 'texture', 'rich', 'flavor', 'found', "'s", 'creamy', 'delicious', 'award-winning', 'made', 'two', 'milk', 'cream', 'pasture-raised', 'perfect', 'coffee', 'tea', 'hot', 'chocolate', 'elevate', 'favorite', 'soups', 'recipes', 'almost', 'always', 'substitute', 'want', 'add', 'extra', 'creaminess', 'pint', '16', 'heavy', 'whipping', 'contains', 'minimum', '36', 'butterfat', 'adds', 'richness', 'desserts', 'sauces', 'whips', 'easily', 'soft', 'peaks', 'deliciously', 'tops', 'morning', 'creams', 'ca', "n't", 'beat', 'folgers', 'gourmet', 'selections', 'lively', 'colombian', 'k-cup', 'keurig', 'brewers', '12', 'count', 'packaging', 'may', 'vary', 'k-cups', 'lactaid', 'reduced', 'fat', '64', 'fl', 'won', 'believe', 'it', '100', 'lactose', 'free', 'essential', 'nutrients', 'like', 'calcium', 'protein', 'vitamin', 'taste', 'buds', 'tummy', '

In [ ]:
# model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0)
# model_dbow.build_vocab([x for x in tqdm(tagged.values)])

In [110]:
class LabeledLineSentence(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for uid, line in enumerate(open(filename)):
            yield LabeledSentence(words=line.split(), labels=['SENT_%s' % uid])

In [128]:
tagged=[]
for i in df['all_info_combined_clean'].values:
    tagged.append(LabeledLineSentence(i))
print(len(tagged))

13615


13615

In [20]:
model.most_similar("energy")

/Users/valamani/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('confidence', 0.3666124939918518),
 ('nutritious', 0.36585575342178345),
 ('immunity', 0.34897926449775696),
 ('brain', 0.3487280011177063),
 ('restful', 0.33978715538978577),
 ('freedom', 0.3392595052719116),
 ('fix', 0.3287910223007202),
 ('caffeine', 0.32371455430984497),
 ('bedtime', 0.3207593858242035),
 ('mornings', 0.3166867196559906)]

In [ ]:
recommender = pairwise_distances(sparse_pivot,metric='cosine')

In [ ]:
recommender_df[["Stacy's Simply Naked Bagel Chips, 7 oz Bag"]]

In [ ]:
recommender_df = pd.DataFrame(recommender, columns=df2.index, index=df2.index)

In [ ]:
df.loc[df['name'].str.contains('Bolthouse'), 'name']

In [ ]:
search = "Bolthouse Farms Green Goodness - 52oz"

for title in df.loc[df['name'].str.contains(search), 'name']:
    print(title)
#     print(recommender_df[title].sort_values()[1:10])
    recommendation = recommender_df[title].sort_values()[1:10]
    print('')
    print('*******************************************************************************************')
    print('')
recom = pd.DataFrame(recommendation)    
recoms = pd.merge(df, recom, on = 'name')
recommended_cat = df.loc[df['name'].str.contains(search),'category'].head(1).values[0]
final = recoms[recoms['category']== recommended_cat]
final[['name','brand', 'store','price','category']]

In [ ]:
recommendation

In [ ]:
recoms
recom = pd.merge(df, recommendation, on = 'name')


In [ ]:
pd.DataFrame(recommendation)

In [ ]:
df[df['name'] == ]

In [ ]:
recommended_category

In [ ]:
search ='Red Bull'
df.loc[df['name'].str.contains(search), 'name']

In [ ]:
recommender_df['Red Bull Energy Drink - 4pk/12 fl oz Cans'].sort_values[0:10]